In [20]:
from constant import invalid_relations_set

from REL.db.generic import GenericLookup


In [21]:
def Map(head, relations, tail, top_first=True, best_scores = True):
#     if head == None or tail == None or relations == None:
#         print ('choice1')
#         return {}
#     head_p_e_m = emb.wiki(str(head), 'wiki')
#     if head_p_e_m is None:
#         print ('choice2')
#         head_p_e_m = head
#     else:
#         print ('choice3')
#         head_p_e_m = head_p_e_m[0][0]   
#     tail_p_e_m = emb.wiki(str(tail), 'wiki')
#     if tail_p_e_m is None:
#         print ('choice4')
#         tail_p_e_m = tail
#     else:
#         print ('choice5')
#         tail_p_e_m = tail_p_e_m[0][0]
     


#     valid_relations = [ r for r in relations if r not in invalid_relations_set and r.isalpha() and len(r) > 1 ]
#     if len(valid_relations) == 0:
#         print ('choice6')
#         return {}

    head_p_e_m = head
    tail_p_e_m = tail
    valid_relations = [ r for r in relations if r not in invalid_relations_set and r.isalpha() and len(r) > 1 ]
    return { 'h': head_p_e_m, 't': tail_p_e_m, 'r': '_'.join(valid_relations)  }

In [22]:
# head ='The invention' 
# relations = ['relates']
# tail = 'a roadable aircraft vehicle'
# triple_dict=Map(head,relations,tail)

# print ("the head: %s , the relations: %s, the tails: %s" %(triple_dict['h'],triple_dict['r'],triple_dict['t']))
# emb.wiki('magnetic bearings','wiki')

In [23]:
import en_core_web_md
from process import parse_sentence
from tqdm import tqdm
from transformers import AutoTokenizer, BertModel, GPT2Model
from mapper import Map, deduplication
nlp = en_core_web_md.load()
import torch
import numpy as np
from utils import compress_attention, create_mapping, BFS, build_graph, is_word
from multiprocessing import Pool
import spacy
import en_core_web_md
import torch
from transformers import AutoTokenizer, BertModel, GPT2Model
from constant import invalid_relations_set
from utils import compress_attention,build_graph

In [24]:
language_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(language_model)
encoder = BertModel.from_pretrained(language_model)
encoder.eval()
use_cuda =False
threshold = 0.01
# def process_matrix(attentions, layer_idx = -1, head_num = 0, avg_head=False, trim=True, use_cuda=True):
#     if avg_head:
#         if use_cuda:
#             attn =  torch.mean(attentions[0][layer_idx], 0).cpu()
#         else:
#             attn = torch.mean(attentions[0][layer_idx], 0)
#         attention_matrix = attn.detach().numpy()
#     else:
#         attn = attentions[0][layer_idx][head_num]
#         if use_cuda:
#             attn = attn.cpu()
#         attention_matrix = attn.detach().numpy()

#     attention_matrix = attention_matrix[1:-1, 1:-1]

#     return attention_matrix
with open ('examples/patent_example.txt','r') as f:
    num_file = sum([1 for i in open('examples/patent_example.txt', "r")])
    #tqdm summmarization
    for idx, line in enumerate(tqdm (f,total=num_file)):
        sentence  = line.strip()
#         if len(sentence):
#             valid_triplets = []
#             for sent in nlp(sentence).sents:
#                 sentence= sent.text
#                 tokenizer_name = str(tokenizer.__str__)
#                 inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)

#                 with torch.no_grad():
#                     if use_cuda:
#                         for key in inputs.keys():
#                             inputs[key] = inputs[key].cuda()
#                     outputs = encoder(**inputs, output_attentions=True)
#                 trim = True
#                 if 'GPT2' in tokenizer_name:
#                     trim  = False

#                 '''
#                 Use average of last layer attention : page 6, section 3.1.2
#                 '''
#                 attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)

#                 merged_attention = compress_attention(attention, tokenid2word_mapping)
#                 attn_graph = build_graph(merged_attention)
#             print ('attention:{}'.format(attention))
#             print ('merged_attention:{}'.format(merged_attention))
#             print ("attn_graph:{}".format(attn_graph))
#             print ("attention_shape:{0},merged_attention_shape:{1},attention_graph_shape:{2}".\
#                    format(attention.shape,              
#                           merged_attention.shape,
#                          len(attn_graph)))

        valid_triplets = []
        count = 0
        doc = nlp(sentence)
        print ("count:{}".format(count))


        for sent in nlp(sentence).sents:
            print ("sent:{}".format(sent))
            print ("sent start :%s sent end :%s"% (sent.start, sent.end))
        start_chunk = []
        end_chunk = []
        noun_chunks = []

        for chunk in doc.noun_chunks:
            noun_chunks.append(chunk.text)
            start_chunk.append(chunk.start)
            end_chunk.append(chunk.end)
        chunk_id = 0
        token2id ={ }
        mode = 1
        sentence_mapping = []

        #### modification starts below,
        ### token2id [sentence_mapping[-1]]= len(token2id) --->token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
        ### because token2id is a dictionary, when you len it, it will automatically remove the duplicated keys, this will
        ### cause problems when the words occurs more than once.

        for idx,token in enumerate (doc):
            if idx in start_chunk:
                mode= 1
                sentence_mapping.append (noun_chunks[chunk_id])
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)
                chunk_id +=1 
            if idx in end_chunk:
                mode = 0
                
            if mode == 0:
                sentence_mapping.append (token.text)
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)

        ### modication ends here 
                
        print ('sentence_mapping:{}'.format (sentence_mapping))
        
        token_ids = []
        tokenid2word_mapping = []
        count = 0
        for token in sentence_mapping:
            subtoken_ids = tokenizer(str(token), add_special_tokens=False)['input_ids']
            tokenid2word_mapping += [ token2id[token] ]*len(subtoken_ids)  # have a look at the following sample result and you will know.
            token_ids += subtoken_ids
#             print ("token_ids:{}".format(token_ids))
#             print ("tokenid2word_mapping:{}".format(tokenid2word_mapping))
#             print ("subtoken_ids:{}".format (subtoken_ids))
#             print ("----next round----")
#             count +=1
#             if count ==3 :
#                 break

                
        print ("noun_chunks:{}".format(noun_chunks))
        print ("start_chunk:{}".format(start_chunk))
        print ("end_chunk:{}".format(end_chunk))
        print ("token2id:{}".format(token2id))
        print ("token_ids:{}".format(token_ids))
        print ("tokenid2word_mapping:{}".format(tokenid2word_mapping))
        print ("subtoken_ids:{}".format (subtoken_ids))
        tokenizer_name = str(tokenizer.__str__)

        outputs = {
            'input_ids': [tokenizer.cls_token_id] + token_ids + [tokenizer.sep_token_id],
            'attention_mask': [1]*(len(token_ids)+2),
            'token_type_ids': [0]*(len(token_ids)+2)     }

        for key, value in outputs.items():
            outputs[key] = torch.from_numpy(np.array(value)).long().unsqueeze(0)
        print("outputs:{}".format(outputs))
        print ("len(tokenids):{}".format(len(token_ids))) #equals to the length of original attn matrix len# 


#         print ("outputs_encoder:{}".format(outputs_encoder))
#         print ("outputs_encoder[2]:{}".format(outputs_encoder[2]))        


        tail_head_pairs = []
#should starts from here
# add'same sentence ' limit here
#         i =0
#         for sent in nlp(sentence).sents:
#             i= i+1
#             if i==3:
#                 break
#             for (head,start_number) in zip(noun_chunks,start_chunk):
#                 print ('sent.start:%s, sent.end: %s, head: %s, start_number: %s' % (sent.start,sent.end,head,start_number))
#                 if start_number < sent.end and start_number >= sent.start:
#                     for (tail,end_number) in zip(noun_chunks,end_chunk):
#                         print ('sent.start:%s, sent.end: %s, tail: %s, end_number: %s' % (sent.start,sent.end,tail,end_number))
#                         if end_number > sent.end or end_number < sent.start or start_number>end_number:
#                             print ('continued')
#                             continue
#                         if head != tail:
#                             print ('succeed')
#                             tail_head_pairs.append((token2id[head], token2id[tail]))

# print (tail_head_pairs)
##should ends here

#write a function.#



#         id2token = { value: key for key, value in token2id.items()}
        #### modification starts below,
        ### id2token = { value: key for key, value in token2id.items()} --->id2token = {key:value for key,value in enumerate(sentence_mapping)}
        ### because in token2id dictionary, there is no duplicate tokens, so it won't be reflect the positions with these tokens
        ### but the later algorithm needs all positions 
        id2token = {key:value for key,value in enumerate(sentence_mapping)}               
                
            

100%|██████████| 1/1 [00:00<00:00, 12.69it/s]

count:0
sent:The invention relates to a roadable aircraft vehicle and related systems.
sent start :0 sent end :12
sent:An example roadable aircraft vehicle includes a vehicle drive system including an engine and gearbox selectively engageable with an automotive driveline  and at least one propeller , a user interface including a display  for controlling the drive system  in an automotive mode including a steering wheel and in a flight mode including a control stick , a control system for switching between the flight mode and the automotive mode, and a system  for locking the propeller during the automotive mode.
sent start :12 sent end :98
sent:The invention also relates to aircraft systems and elements such as an airfoil  having a nominal profile, a folding wing , and an occupant crash protection system for an aircraft .
sent start :98 sent end :131
sentence_mapping:['The invention', 'relates', 'to', 'a roadable aircraft vehicle', 'and', 'related systems', '.', 'An example roadable ai

In [16]:
print ('token2id:{'The invention': 57, 'relates': 59, 'to': 60, 'a roadable aircraft vehicle': 3, 'and': 73, 'related systems': 5, '.': 77, 'An example roadable aircraft vehicle': 7, 'includes': 8, 'a vehicle drive system': 9, 'including': 37, 'an engine': 11, 'gearbox': 13, 'selectively': 14, 'engageable': 15, 'with': 16, 'an automotive driveline': 17, ' ': 67, 'at least one propeller': 20, ',': 72, 'a user interface': 22, 'a display': 24, 'for': 75, 'controlling': 27, 'the drive system': 28, 'in': 35, 'an automotive mode': 31, 'a steering wheel': 33, 'a flight mode': 36, 'a control stick': 38, 'a control system': 40, 'switching': 42, 'between': 43, 'the flight mode': 44, 'the automotive mode': 55, 'a system': 49, 'locking': 52, 'the propeller': 53, 'during': 54, 'also': 58, 'aircraft systems': 61, 'elements': 63, 'such': 64, 'as': 65, 'an airfoil': 66, 'having': 68, 'a nominal profile': 69, 'a folding wing': 71, 'an occupant crash protection system': 74, 'an aircraft': 76}')

SyntaxError: invalid syntax (<ipython-input-16-f02f5ff455c4>, line 1)

In [48]:
id2noun_chunk_token=dict()
for key,value in  id2token.items():
    if value in noun_chunks:
        id2noun_chunk_token[i]=key
        i=i+1
print (id2noun_chunk_token)

def head_tail_pairs_append(head_tail_pairs,sent,id2noun_chunk_token,start_chunk,token2id):
    print ('sent.start:%s, sent.end:%s'%(sent.start,sent.end))
    for i1,start1 in enumerate(start_chunk):
        print ('i1:%s, start1 :%s' %(i1, start1))
        if start1 < sent.start:
            continue
        if start1 >=sent.end:
            break
        for i2,start2 in enumerate(start_chunk):
#             print ("i2:%s, start2 :%s"%(i2, start2))
            if start2<start1 :
                continue
            if start2>= sent.end: 
                break
            if start1!=start2:
#                 id2token[i1]
#                 id2token[i2]
#                 print ("succeed, start1 :%s,start2:%s"%(noun_chunks[i1],noun_chunks[i2]))
                print ('sc')
                head_tail_pairs.append((id2noun_chunk_token[i1],id2noun_chunk_token[i2]))
    return head_tail_pairs

{0: 0, 1: 3, 2: 5, 3: 7, 4: 9, 5: 11, 6: 13, 7: 17, 8: 20, 9: 22, 10: 24, 11: 28, 12: 31, 13: 33, 14: 36, 15: 38, 16: 40, 17: 44, 18: 46, 19: 49, 20: 53, 21: 55, 22: 57, 23: 61, 24: 63, 25: 66, 26: 69, 27: 71, 28: 74, 29: 76}


In [49]:
head_tail_pairs=[]
print (head_tail_pairs)
with open ('examples/patent_example.txt','r') as f:
    num_file = sum([1 for i in open('examples/patent_example.txt', "r")])
    #tqdm summmarization
    for idx, line in enumerate(tqdm (f,total=num_file)):
        sentence  = line.strip()
        doc = nlp(sentence)

        i=0
        for sent in nlp(sentence).sents: 
            print(sent)
            head_tail_pairs=head_tail_pairs_append(head_tail_pairs,sent,id2noun_chunk_token,start_chunk,token2id)
        print (head_tail_pairs)
print (id2token)

100%|██████████| 1/1 [00:00<00:00, 11.02it/s]

[]
The invention relates to a roadable aircraft vehicle and related systems.
sent.start:0, sent.end:12
i1:0, start1 :0
sc
sc
i1:1, start1 :4
sc
i1:2, start1 :9
i1:3, start1 :12
An example roadable aircraft vehicle includes a vehicle drive system including an engine and gearbox selectively engageable with an automotive driveline  and at least one propeller , a user interface including a display  for controlling the drive system  in an automotive mode including a steering wheel and in a flight mode including a control stick , a control system for switching between the flight mode and the automotive mode, and a system  for locking the propeller during the automotive mode.
sent.start:12, sent.end:98
i1:0, start1 :0
i1:1, start1 :4
i1:2, start1 :9
i1:3, start1 :12
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
i1:4, start1 :18
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
i1:5, start1 :23
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
sc
i1:6, start1 :26
sc
sc
sc
sc
sc
sc
sc
sc
sc

In [30]:
print ('token2id:{'The invention': 57, 'relates': 59, 'to': 60, 'a roadable aircraft vehicle': 3, 'and': 73, 'related systems': 5, '.': 77, 'An example roadable aircraft vehicle': 7, 'includes': 8, 'a vehicle drive system': 9, 'including': 37, 'an engine': 11, 'gearbox': 13, 'selectively': 14, 'engageable': 15, 'with': 16, 'an automotive driveline': 17, ' ': 67, 'at least one propeller': 20, ',': 72, 'a user interface': 22, 'a display': 24, 'for': 75, 'controlling': 27, 'the drive system': 28, 'in': 35, 'an automotive mode': 31, 'a steering wheel': 33, 'a flight mode': 36, 'a control stick': 38, 'a control system': 40, 'switching': 42, 'between': 43, 'the flight mode': 44, 'the automotive mode': 55, 'a system': 49, 'locking': 52, 'the propeller': 53, 'during': 54, 'also': 58, 'aircraft systems': 61, 'elements': 63, 'such': 64, 'as': 65, 'an airfoil': 66, 'having': 68, 'a nominal profile': 69, 'a folding wing': 71, 'an occupant crash protection system': 74, 'an aircraft': 76}')

SyntaxError: invalid syntax (<ipython-input-30-f02f5ff455c4>, line 1)

In [88]:
print |(noun_chunks:['The invention', 'a roadable aircraft vehicle', 'related systems', 'An example roadable aircraft vehicle', 'a vehicle drive system', 'an engine', 'gearbox', 'an automotive driveline', 'at least one propeller', 'a user interface', 'a display', 'the drive system', 'an automotive mode', 'a steering wheel', 'a flight mode', 'a control stick', 'a control system', 'the flight mode', 'the automotive mode', 'a system', 'the propeller', 'the automotive mode', 'The invention', 'aircraft systems', 'elements', 'an airfoil', 'a nominal profile', 'a folding wing', 'an occupant crash protection system', 'an aircraft'])

SyntaxError: invalid syntax (<ipython-input-88-b2c877e77650>, line 1)

In [1]:
from transformers import AutoTokenizer, BertModel, GPT2Model
language_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(language_model)

token ='a roadable aircraft vehicle'  # it's because roadable is not a real word in dic, so splitted into 2 parts#

## how to deal with this ##??
subtoken_ids = tokenizer(str(token), add_special_tokens=False)['input_ids']


In [2]:
print (subtoken_ids)

[106, 6229, 318, 12118, 4818]


In [10]:
token = 'a roadable opportunity'
subtoken_ids = tokenizer(str(token), add_special_tokens=False) ['input_ids']
print (subtoken_ids)

[106, 6229, 318, 7336]


In [16]:
token ='a926'
subtoken_ids = tokenizer(str(token), add_special_tokens=False) ['input_ids']
print (subtoken_ids)

[106, 6386, 30142]
